In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
m = 100
x = tf.constant(np.random.uniform(size =(2,m)), dtype = tf.float32)
y = tf.constant(np.random.randint(2,size =(1,m)), dtype = tf.float32)

In [ ]:
w1 = tf.constant(np.random.uniform(size =(4, 2)), dtype = tf.float32)
b1 = tf.constant(np.zeros(shape = (4, 1)), dtype = tf.float32)

w2 = tf.constant(np.random.uniform(size =(4, 4)), dtype = tf.float32)
b2 = tf.constant(np.zeros(shape = (4, 1)), dtype = tf.float32)

w3 = tf.constant(np.random.uniform(size =(1, 4)), dtype = tf.float32)
b3 = tf.constant(np.zeros(shape = (1, 1)), dtype = tf.float32)

# class Weights():
#     def __init__(self, Nunits, kernel_fn = 'random', bias_fn = 'zeros'):
#         self.Nunits = Nunits
#         self.init_fn = init_fn
        
#     def __call__(self):
#         self.weights = []
#         for u in range(1,len(Nunits)):
#             w = self.random((Nunits[u],Nunits[u - 1]))
#             b = self.zero((Nunits[u],1))
#             self.weights.append(w)
#             self.weights.append(b)
            
#     def random(self, shape):
#         return tf.random.uniform(shape = shape)
    
#     def zero(self, shape):
#         return tf.zeros(shape = shape)

In [ ]:
class Derivative():
    def __init__(self, function):
        self.function = function
    
    def __call__(self, *args):
        return getattr(Derivative, self.function)(*args)
    
    def sigmoid(A):
        return A * (1 - A)
    
    def tanh(A):
        return 1 - A ** 2
    
    def binary_crossentropy(Y, A):
        return - (Y / A) + ((1 - Y) / (1 - A))

# test_in = tf.constant(np.random.uniform(size =(4, 2)), dtype = tf.float32)    
# Derivative('tanh')(test_in)

In [ ]:
class Dense():
    def __init__(self, units, activation, weights, bias):
        self.units = units
        self.activation = activation
        self.weights = weights
        self.bias = bias
        
    def __call__(self, X):
        return self.forward_step(X)
        
    def forward_step(self, X):
        self.Z = tf.matmul(self.weights, X) + self.bias
        
        if self.activation == 'sigmoid':
            self.A = tf.math.sigmoid(self.Z)
            return self.A
        elif self.activation == 'tanh':
            self.A = tf.math.tanh(self.Z)
            return self.A
        
    def backward_step(self, dA, A_prev, m):
        self.dz = dA * Derivative(self.activation)(self.A)
        self.dw = tf.matmul(self.dz, A_prev, transpose_b = True) / m
        self.db = tf.reduce_sum(self.dz, axis = 1, keepdims = True) / m
        return tf.matmul(self.weights, self.dz, transpose_a = True)
            
        
    def update_weights_and_biases(self, lr):
        self.weights = self.weights - lr * self.dw
        self.bias = self.bias - lr * self.db
            
    def get_z(self):
        return self.Z
    
    def get_weights(self):
        return self.weights, self.bias
    
# h1 = Dense(units = 4, activation = 'tanh', weights = w1, bias = b1)
# h2 = Dense(units = 4, activation = 'tanh', weights = w2, bias = b2)
# o1 = Dense(units = 1, activation = 'sigmoid', weights = w3, bias = b3)            
# o1(h2(h1(x)))

In [ ]:
class Model():
    def __init__(self,layers, lossfn, lr):
        self.lossfn = lossfn
        self.lr = lr
        self.layers = layers
        
    def __call__(self,X):
        return self.forward_propagation(X)
        
    def forward_propagation(self, A):
        self.all_activations = []
        self.all_activations.append(A)
        for layer in self.layers:
            A = layer(A)
            self.all_activations.append(A)
        return self.all_activations[-1]
    
    def backward_propagation(self, Y):
        self.m = Y.shape[1]
        dA = Derivative(self.lossfn)(Y, self.all_activations[-1])
        for l in reversed(range(len(self.layers))):
            dA = self.layers[l].backward_step(dA, self.all_activations[l], self.m)
            self.layers[l].update_weights_and_biases(self.lr)

# h1 = Dense(units = 4, activation = 'tanh', weights = w1, bias = b1)
# h2 = Dense(units = 4, activation = 'tanh', weights = w2, bias = b2)
# o1 = Dense(units = 1, activation = 'sigmoid', weights = w3, bias = b3)            
# layers = [h1, h2, o1]
# model = Model(layers, 'binary_crossentropy', 0.1)
# model(x)     
# model.backward_propagation(y)

In [ ]:
h1 = Dense(units = 4, activation = 'tanh', weights = w1, bias = b1)
h2 = Dense(units = 4, activation = 'tanh', weights = w2, bias = b2)
o1 = Dense(units = 1, activation = 'sigmoid', weights = w3, bias = b3)            
layers = [h1, h2, o1]
model = Model(layers, 'binary_crossentropy', 0.1)

EPOCHS = 10
for i in range(EPOCHS):
    y_hat = model(x)
    loss = tf.squeeze((tf.matmul(y,tf.math.log(y_hat), transpose_b = True) + tf.matmul(1 - y,tf.math.log(1 - y_hat), transpose_b = True)) * (-1/y.shape[1]))
    model.backward_propagation(y)
    print('Epoch: {}   loss: {}'.format(i+1, loss))

In [ ]:
# using keras to get same results
def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units = 4,activation = 'tanh',  name = 'd1', input_shape = (x.shape[0],)))
    model.add(tf.keras.layers.Dense(units = 4,activation = 'tanh',  name = 'd2'))
    model.add(tf.keras.layers.Dense(units = 1,activation = 'sigmoid',  name = 'o1'))
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1),
                loss='binary_crossentropy', metrics = ['accuracy'])
    
    return model
model = create_model()

d1 = model.get_layer('d1')
d1_weights = [tf.constant(tf.transpose(w1), dtype = tf.float32), tf.constant(tf.squeeze(b1), dtype = tf.float32)]
d1.set_weights(d1_weights)

d2 = model.get_layer('d2')
d2_weights = [tf.constant(tf.transpose(w2), dtype = tf.float32), tf.constant(tf.squeeze(b2), dtype = tf.float32)]
d2.set_weights(d2_weights)


o1 = model.get_layer('o1')
o1_weights = [tf.constant(tf.transpose(w3), dtype = tf.float32), tf.constant(tf.squeeze(b3, axis = 1), dtype = tf.float32)]
o1.set_weights(o1_weights)

xt = tf.transpose(x)
yt = tf.transpose(y)

In [ ]:
model.fit(xt, yt, epochs = 10, batch_size = 1000)


In [ ]:
model.weights[1]